In [ ]:
import tensorflow as tf
import numpy as np
import os
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt 

import os
import zipfile
import requests
import numpy as np
import tensorflow as tf
from tqdm import tqdm
from pathlib import Path
import sys 

import urllib.request
import gzip
import pickle

rand = np.random.RandomState(42)

from tensorflow.python.client import device_lib 
print(device_lib.list_local_devices())

In [ ]:

cwd = os.getcwd()
parts = cwd.split(os.sep)
pyda_index = parts.index("PyDA") + 1
pyda_path = os.sep.join(parts[:pyda_index])
sys.path.append(pyda_path)

from TL_models.deep.DANN import DANN_model
from TL_models.deep.DAN import DAN_model
from TL_models.deep.CDAN import CDAN_model
from TL_models.deep.JAN import JAN_model

In [ ]:

def load_mnist():
    """Loads the MNIST dataset, pads to 32x32, and converts to 3-channel."""
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    
    x_train = (x_train.astype("float32")) 
    x_test = (x_test.astype("float32")) 
    x_train = np.expand_dims(x_train, axis=-1)
    x_test = np.expand_dims(x_test, axis=-1)

    # grayscale to 3-channel RGB (28,28,3)
    x_train = np.repeat(x_train, 3, axis=-1)
    x_test = np.repeat(x_test, 3, axis=-1)

    return (x_train, y_train), (x_test, y_test)

(X_mnist_train, y_mnist_train), (X_mnist_test, y_mnist_test) = load_mnist()

print(f"Training shape: {X_mnist_train.shape}, testing shape: {X_mnist_test.shape}")


In [ ]:

URL = "https://github.com/VanushVaswani/keras_mnistm/releases/download/1.0/keras_mnistm.pkl.gz"
FILE_PATH = "keras_mnistm.pkl.gz"
EXTRACTED_FILE_PATH = "keras_mnistm.pkl"

def download_mnistm():
    if not os.path.exists(FILE_PATH):
        print("Downloading MNIST-M dataset...")
        urllib.request.urlretrieve(URL, FILE_PATH)
        print("Download complete.")

def load_mnistm():
    with gzip.open(FILE_PATH, "rb") as f:
        mnistm_data = pickle.load(f, encoding="latin1")
    return mnistm_data

download_mnistm()
mnistm_data = load_mnistm()

X_mnistm_train = np.array(mnistm_data["train"]).astype(np.float32) 
X_mnistm_test = np.array(mnistm_data["test"]).astype(np.float32) 
y_mnistm_train, y_mnistm_test = y_mnist_train, y_mnist_test

np.savez_compressed('mnist_train.npz', X = X_mnist_train, y = y_mnist_train)
np.savez_compressed('mnist_test.npz', X = X_mnist_test, y = y_mnist_test)
np.savez_compressed("mnistm_train.npz", X=X_mnistm_train, y=y_mnist_train)
np.savez_compressed("mnistm_test.npz", X=X_mnistm_test, y=y_mnist_test)


In [ ]:
# ### Uncomment if running this multiple times ###

# mnist_train = np.load('mnist_train.npz')
# mnist_test = np.load('mnist_test.npz')
# mnistm_train = np.load('mnistm_train.npz')
# mnistm_test = np.load('mnistm_test.npz')

# X_mnist_train, y_mnist_train = mnist_train['X'], mnist_train['y']
# X_mnist_test, y_mnist_test = mnist_test['X'], mnist_test['y']
# X_mnistm_train, y_mnistm_train = mnistm_train['X'], mnistm_train['y']
# X_mnistm_test, y_mnistm_test = mnistm_test['X'], mnistm_test['y']

# print("MNIST Train Shape:", X_mnist_train.shape, "MNIST-M Train Shape:", X_mnistm_train.shape)
# print("MNIST Test Shape:", X_mnist_test.shape, "MNIST-M Test Shape:", X_mnistm_test.shape)


In [ ]:
##min-max normalisation for visualusation
X_mnist_train = (X_mnist_train) / 255
X_mnist_test = (X_mnist_test) / 255

X_mnistm_train = (X_mnistm_train) / 255
X_mnistm_test = (X_mnistm_test) / 255

num_images = 5  

fig, axes = plt.subplots(1, 5, figsize=(15, 6))  #
axes = axes.flatten()  
for i in range(num_images):
    ax = axes[i]
    ax.imshow(X_mnist_train[i], cmap='gray')  
    ax.axis('off')  
    ax.set_title(y_mnist_train[i])

plt.tight_layout()
plt.show()

fig, axes = plt.subplots(1, 5, figsize=(15, 6)) 
axes = axes.flatten()  
for i in range(num_images):
    ax = axes[i]
    ax.imshow(X_mnistm_train[i], cmap='gray')  
    ax.axis('off')  
    ax.set_title(y_mnistm_train[i])
plt.tight_layout()
plt.show()

In [ ]:
#normalise for training

##z-score normalisation
mnist_mu = np.mean(X_mnist_train[...,0])
mnist_std = np.std(X_mnist_train[...,0])
X_mnist_train = (X_mnist_train - mnist_mu) / mnist_std
X_mnist_test = (X_mnist_test - mnist_mu) / mnist_std

mnistm_mu = np.mean(X_mnistm_train, axis=(0,1,2))
mnistm_std = np.std(X_mnistm_train, axis=(0,1,2))
X_mnistm_train = (X_mnistm_train - mnistm_mu) / mnistm_std
X_mnistm_test = (X_mnistm_test - mnistm_mu) / mnistm_std



In [ ]:
###standard neural network 
LR = 1e-3
EPOCHS = 100

params={'feat_conv_layers': [[32,(5,5)], [48,(5,5)]],#[filters, kernel]
        'feat_fc_layers': [],
        'class_layers': [100, 100],
        'disc_layers':[100],
        'input_dim': 3,
        'output_size': 10,
        'drop_rate': 0.25,
        'reg': 1e-3,
        'entropy': 0,
        'BN': True,
        'lr': LR,
        'pool_size':(2,2),
        'stride': (1,1)}

cnn = DANN_model(params = params, optimiser = tf.keras.optimizers.SGD())

#pretrain = True sets lmda = 0
cnn.fit(X_mnist_train, y_mnist_train, X_mnistm_train,
          epochs=EPOCHS, batch_size = 32, print_training=True, pretrain=True)

In [ ]:
source_f1, source_acc = cnn.get_performance(X_mnist_test, y_mnist_test)
target_f1, target_acc = cnn.get_performance(X_mnistm_test, y_mnistm_test)
print(f'Source test f1 {source_f1}, target test f1 {target_f1}')
print(f'Target test acc {source_acc}, target test acc {target_acc}')

cnn.plot_feature([X_mnist_test[:500], y_mnist_test[:500], X_mnistm_test[:500], y_mnistm_test[:500]],
                   colour='domain', name=None, feature_reduction='t-SNE')

In [ ]:


params={'feat_conv_layers': [[32,(5,5)], [48,(5,5)]],#[filters, kernel]
        'feat_fc_layers': [],
        'class_layers': [100, 100],
        'input_dim': 3,
        'output_size': 10,
        'drop_rate': 0.25,
        'reg': 1e-3,
        'entropy': 0,
        'BN': True,
        'lr': LR,
        'pool_size':(2,2),
        'stride': (1,1), 
        'length scales':None}

dan = DAN_model(params = params, optimiser = tf.keras.optimizers.SGD())
dan.update=True
dan.fit(X_mnist_train, y_mnist_train, X_mnistm_train,
          epochs=EPOCHS, batch_size = 32, print_training=True, pretrain=False, lmda = 1.0) #lmda=10

In [ ]:
source_f1, source_acc = dan.get_performance(X_mnist_test, y_mnist_test)
target_f1, target_acc = dan.get_performance(X_mnistm_test, y_mnistm_test)
print(f'Source test f1 {source_f1}, target test f1 {target_f1}')
print(f'Source test f1 {source_acc}, target test acc {target_acc}')

dan.plot_feature([X_mnist_test[:500], y_mnist_test[:500], X_mnistm_test[:500], y_mnistm_test[:500]],
                   colour='domain', name=None, feature_reduction='t-SNE')

In [ ]:
params={'feat_conv_layers': [[32,(5,5)], [48,(5,5)]],#[filters, kernel]
        'feat_fc_layers': [],
        'class_layers': [100, 100],
        'disc_layers':[100],
        'input_dim': 3,
        'output_size': 10,
        'drop_rate': 0.25,
        'reg': 1e-3,
        'entropy': 0,
        'BN': True,
        'lr': LR,
        'pool_size':(2,2),
        'stride': (1,1), 
        'length scales':'median'}#WARNING ONLY median is set up for JAN
                                 #requires a list of length scales for each layer
jan = JAN_model(params = params, optimiser = tf.keras.optimizers.SGD())
jan.update=True
jan.fit(X_mnist_train, y_mnist_train, X_mnistm_train,
          epochs=EPOCHS, batch_size = 32, print_training=True, pretrain=False, lmda = 1e6) #

In [ ]:
source_f1, source_acc = jan.get_performance(X_mnist_test, y_mnist_test)
target_f1, target_acc = jan.get_performance(X_mnistm_test, y_mnistm_test)
print(f'Source test f1 {source_f1}, target test f1 {target_f1}')
print(f'Target test acc {source_acc}, target test acc {target_acc}')

jan.plot_feature([X_mnist_test[:500], y_mnist_test[:500], X_mnistm_test[:500], y_mnistm_test[:500]],
                   colour='domain', name=None, feature_reduction='t-SNE')

In [ ]:

params={'feat_conv_layers': [[32,(5,5)], [48,(5,5)]],#[filters, kernel]
        'feat_fc_layers': [],
        'class_layers': [100, 100],
        'disc_layers':[100],
        'input_dim': 3,
        'output_size': 10,
        'drop_rate': 0.25,
        'reg': 1e-3,
        'entropy': 0,
        'BN': True,
        'pool_size':(2,2),
        'lr': LR,
        'stride': (1,1)}

dann = DANN_model(params = params, optimiser = tf.keras.optimizers.SGD())

dann.fit(X_mnist_train, y_mnist_train, X_mnistm_train,
          epochs=EPOCHS, batch_size = 32, print_training=True, pretrain=False)

In [ ]:
source_f1, source_acc = dann.get_performance(X_mnist_test, y_mnist_test)
target_f1, target_acc = dann.get_performance(X_mnistm_test, y_mnistm_test)
print(f'Source test f1 {source_f1}, target test f1 {target_f1}')
print(f'Target test acc {source_acc}, target test acc {target_acc}')

dann.plot_feature([X_mnist_test[:500], y_mnist_test[:500], X_mnistm_test[:500], y_mnistm_test[:500]],
                   colour='domain', name=None, feature_reduction='t-SNE')

In [ ]:

cdan = CDAN_model(params = params, optimiser = tf.keras.optimizers.SGD())
cdan.fit(X_mnist_train, y_mnist_train, X_mnistm_train,
          epochs=EPOCHS, batch_size = 32, print_training=True, pretrain=False)

In [ ]:
source_f1, source_acc = cdan.get_performance(X_mnist_test, y_mnist_test)
target_f1, target_acc = cdan.get_performance(X_mnistm_test, y_mnistm_test)
print(f'Source test f1 {source_f1}, target test f1 {target_f1}')
print(f'Target test acc {source_acc}, target test acc {target_acc}')

cdan.plot_feature([X_mnist_test[:500], y_mnist_test[:500], X_mnistm_test[:500], y_mnistm_test[:500]],
                   colour='domain', name=None, feature_reduction='t-SNE')